In [1]:
import os
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
from decimal import Decimal


data = pd.read_pickle('../data/session_data.dataset')
train_data = pd.read_pickle('../data/train_data_005.dataset')
valid_data = pd.read_pickle('../data/valid_data_005.dataset')
test_data = pd.read_pickle('../data/test_data_p2.dataset')
test_data2 = pd.read_pickle('../data/test_data.dataset')
test_data3 = pd.read_pickle('../data/test_data_more.dataset')
test_data2 = pd.concat([test_data2,test_data3]).reset_index(drop=True)
locale2ids = pickle.load(open('../data/local2ids.pkl', 'rb'))
product_id2asin = pickle.load(open('../data/product_id2asin.pkl', 'rb'))
product_asin2id = pickle.load(open('../data/product_asin2id.pkl', 'rb'))
session = data['session'].tolist()
locale = data['locale'].tolist()
valid_session = valid_data['session'].tolist()
valid_locale = valid_data['locale'].tolist()
train_session = train_data['session'].tolist()
train_locale = train_data['locale'].tolist()
test_session = test_data['session'].tolist()
test_locale = test_data['locale'].tolist()
test_session2 = test_data2['session'].tolist()
test_locale2 = test_data2['locale'].tolist()
itemnum = 1410675 + 1
test_time = 1
pro_df = pd.read_csv("../data/products_train.csv", sep=',')
item = pro_df['id']
item = [product_asin2id[x] for x in item]
pro_df['item'] = item

In [2]:
#locale map number
locale_map = {}
temp = 0
for item in list(set(locale)):
    locale_map[item] = temp
    temp += 1

In [3]:
author = pro_df['author'].tolist()
author = [str(x) for x in author]
author_key = list(set(author))
author_number = len(author_key)
author_values = list(range(1,len(author_key)+1))
author_s2n = dict(zip(author_key, author_values))
author_s2n['nan'] = 0
author_map = [{} for i in range(temp)]
author_locale = pro_df['locale'].tolist()
author_item = pro_df['item'].tolist()
author_list = [Counter() for i in range(temp)]
for i in range(len(author)):
    if author_locale[i] in ['FR','ES','IT']:
        continue
    author_map[locale_map[author_locale[i]]][author_item[i]]= author_s2n[author[i]]
    if author_s2n[author[i]] !=0:
        author_list[locale_map[author_locale[i]]][author_s2n[author[i]]] += 1

In [4]:
brand = pro_df['brand'].tolist()
brand = [str(x) for x in brand]
brand_key = list(set(brand))
brand_number = len(brand_key)
brand_values = list(range(1,len(brand_key)+1))
brand_s2n = dict(zip(brand_key, brand_values))
brand_s2n['nan'] = 0
brand_map = [{} for i in range(temp)]
brand_list = [Counter() for i in range(temp)]
brand_locale = pro_df['locale'].tolist()
brand_item = pro_df['item'].tolist()
for i in range(len(brand)):
    if brand_locale[i] in ['FR','ES','IT']:
        continue
    brand_map[locale_map[brand_locale[i]]][brand_item[i]]= brand_s2n[brand[i]]
    if brand_s2n[brand[i]] !=0:
        brand_list[locale_map[brand_locale[i]]][brand_s2n[brand[i]]] += 1

In [5]:
material = pro_df['material'].tolist()
material = [str(x) for x in material]
material_key = list(set(material))
material_number = len(material_key)
material_values = list(range(1,len(material_key)+1))
material_s2n = dict(zip(material_key, material_values))
material_s2n['nan'] = 0
material_map = [{} for i in range(temp)]
material_locale = pro_df['locale'].tolist()
material_item = pro_df['item'].tolist()
for i in range(len(material)):
    if material_locale[i] in ['FR','ES','IT']:
        continue
    material_map[locale_map[material_locale[i]]][material_item[i]]= material_s2n[material[i]]

### session process

In [6]:
for i in tqdm(range(len(session))):
    row = []
    for j in range(len(session[i])):
        if j != 0:
            if session[i][j] == session[i][j-1]:
                continue
        row.append(session[i][j])
    session[i] = row
    
for i in tqdm(range(len(train_session))):
    row = []
    for j in range(len(train_session[i])):
        if j != 0:
            if train_session[i][j] == train_session[i][j-1]:
                continue
        row.append(train_session[i][j])
    train_session[i] = row
    
for i in tqdm(range(len(valid_session))):
    row = []
    for j in range(len(valid_session[i])):
        if j != 0:
            if valid_session[i][j] == valid_session[i][j-1]:
                continue
        row.append(valid_session[i][j])
    valid_session[i] = row

for i in tqdm(range(len(test_session))):
    row = []
    for j in range(len(test_session[i])):
        if j != 0:
            if test_session[i][j] == test_session[i][j-1]:
                continue
        row.append(test_session[i][j])
    test_session[i] = row

for i in tqdm(range(len(test_session2))):
    row = []
    for j in range(len(test_session2[i])):
        if j != 0:
            if test_session2[i][j] == test_session2[i][j-1]:
                continue
        row.append(test_session2[i][j])
    test_session2[i] = row

100%|██████████| 376971/376971 [00:00<00:00, 841993.61it/s]


### 1.matrix

In [7]:
apper = 1
item_next = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    for j in range(len(row)-1):
        item_next[locale_map[locale[i]]][row[j]][row[j+1]] += apper
item_next3 = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    check = Counter()
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] += apper
        item_next3[locale_][row[j]][row[j+3]] += apper


100%|██████████| 3272716/3272716 [00:14<00:00, 228203.90it/s]


In [8]:
apper = 3
for i in tqdm(range(len(test_session))):
    row = test_session[i]
    for j in range(len(row)-1):
        item_next[locale_map[test_locale[i]]][row[j]][row[j+1]] += apper
for i in tqdm(range(len(test_session))):
    check = Counter()
    row = test_session[i]
    locale_ = locale_map[test_locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] += apper
        item_next3[locale_][row[j]][row[j+3]] += apper
apper = 2  
for i in tqdm(range(len(test_session2))):
    row = test_session2[i]
    for j in range(len(row)-1):
        item_next[locale_map[test_locale2[i]]][row[j]][row[j+1]] += apper
for i in tqdm(range(len(test_session2))):
    check = Counter()
    row = test_session2[i]
    locale_ = locale_map[test_locale2[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] += apper
        item_next3[locale_][row[j]][row[j+3]] += apper

100%|██████████| 376971/376971 [00:01<00:00, 329261.05it/s]


### 2 matrix：back

In [9]:
item_next3_back = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    check = Counter()
    apper = 1
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3_back[locale_][row[j+3]][row[j]] += apper
        item_next3_back[locale_][row[j+3]][row[j]] += apper

100%|██████████| 3272716/3272716 [00:14<00:00, 226396.86it/s]


In [10]:
for i in tqdm(range(len(test_session))):
    check = Counter()
    apper = 3
    row = test_session[i]
    locale_ = locale_map[test_locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3_back[locale_][row[j+3]][row[j]] += apper
        item_next3_back[locale_][row[j+3]][row[j]] += apper
        
for i in tqdm(range(len(test_session2))):
    check = Counter()
    apper = 2
    row = test_session2[i]
    locale_ = locale_map[test_locale2[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3_back[locale_][row[j+3]][row[j]] += apper
        item_next3_back[locale_][row[j+3]][row[j]] += apper

100%|██████████| 376971/376971 [00:01<00:00, 326543.40it/s]


### 3.matrix：mix

In [11]:

item_temp = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    check = Counter()
    locale_ = locale_map[locale[i]]
    apper = 1
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] += apper
            item_temp[locale_][row[j+1]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+1]] += apper
        item_temp[locale_][row[j+1]][row[j]] += apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = apper
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] += apper
            item_temp[locale_][row[j+2]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+2]] += apper
        item_temp[locale_][row[j+2]][row[j]] += apper


100%|██████████| 3272716/3272716 [00:40<00:00, 81182.37it/s]


In [12]:
for i in tqdm(range(len(test_session))):
    row = test_session[i]
    check = Counter()
    locale_ = locale_map[test_locale[i]]
    apper = 3
    for j in range(len(row)-1):

        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] += apper
            item_temp[locale_][row[j+1]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+1]] += apper
        item_temp[locale_][row[j+1]][row[j]] += apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = apper
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] += apper
            item_temp[locale_][row[j+2]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+2]] += apper
        item_temp[locale_][row[j+2]][row[j]] += apper

for i in tqdm(range(len(test_session2))):
    row = test_session2[i]
    check = Counter()
    locale_ = locale_map[test_locale2[i]]
    apper = 2
    for j in range(len(row)-1):

        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] += apper
            item_temp[locale_][row[j+1]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+1]] += apper
        item_temp[locale_][row[j+1]][row[j]] += apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = apper
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] += apper
            item_temp[locale_][row[j+2]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+2]] += apper
        item_temp[locale_][row[j+2]][row[j]] += apper

100%|██████████| 376971/376971 [00:03<00:00, 113778.83it/s]


### 4.martix：last time

In [13]:
#train data
temp_list = [1,2,3,5,20]
item_last_time = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    weight = 0.
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        item_last_time[locale_map[locale[i]]][row[-j]][row[-1]] += 1.0*weight

100%|██████████| 3272716/3272716 [00:15<00:00, 213904.05it/s]


### 5.martix：time

In [14]:

item_time = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] += weight

100%|██████████| 3272716/3272716 [01:43<00:00, 31534.12it/s]


In [15]:
for i in tqdm(range(len(test_session))):
    row = test_session[i]
    locale_ = locale_map[test_locale[i]]
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 3
            if k<j:
                weight = 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] += weight
            
for i in tqdm(range(len(test_session2))):
    row = test_session2[i]
    locale_ = locale_map[test_locale2[i]]
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 2
            if k<j:
                weight = 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] += weight

100%|██████████| 376971/376971 [00:07<00:00, 51046.21it/s]


### martix：6.last

In [16]:
item_next_last = [[ Counter() for _ in range(itemnum)] for i in range(3)]

for i in tqdm(range(len(session))):
    row = session[i]
    locale_ = locale_map[locale[i]]
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] += weight

100%|██████████| 3272716/3272716 [00:04<00:00, 744675.70it/s]


### Counter tool

In [17]:
def c_plus(counter, number):
    if number == -1:
        number = 0
        temp = list(counter.values())
        if len(temp) !=0:
            number = max(temp)
    if number !=0:
        ans = dict(zip(counter.keys(),np.array(list(counter.values()))*number))
        ans = Counter(ans)
    else:
        ans = Counter()
    return ans

## test in valid data

In [ ]:
import copy
def mrr_():
    mrr = 0
    check_ = 0
    hit = 0
    jz = []
    for i in tqdm(range(len(valid_session))):
        
        locale_ = locale_map[valid_locale[i]]
        label = valid_session[i][-1]
        prev_items = valid_session[i][:-1]
        prev_items = list(dict.fromkeys(prev_items[::-1]))[::-1]
        output = []
        index  = 999999
        number = 0
        temp = Counter()
        # Eliminate the effect of this session on the matrix
        w_list = [2,3,5]
        apper = 1
        row = valid_session[i]
        weight = 1
        if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
            weight=2
        item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
        for j in range(len(row)-1):
            item_next[locale_][row[j]][row[j+1]] -= apper
        check = Counter()
        for j in range(len(row)-3):
            if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
                continue
            check[(row[j],row[j+3])] = 1
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
                item_next3[locale_][row[j]][row[j+3]] -= apper
                item_next3_back[locale_][row[j+3]][row[j]] -= apper
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        check = Counter()
        for j in range(len(row)-1):
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
                item_temp[locale_][row[j]][row[j+1]] -= apper
                item_temp[locale_][row[j+1]][row[j]] -= apper
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        for j in range(len(row)-2):
            if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
                continue
            check[(row[j],row[j+2])] = 1
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
                item_temp[locale_][row[j]][row[j+2]] -= apper
                item_temp[locale_][row[j+2]][row[j]] -= apper
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        for j in range(len(row)):
            for k in range(len(row)):
                if j == k:
                    continue
                weight = 1
                if k<j:
                    weight *= 0.8
                if abs(k-j)>5:
                    weight *= 0.1
                elif abs(k-j)>=3:
                    weight *= 0.3
                elif abs(k-j)>=2:
                    weight *= 0.5
                if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                    weight *=2
                item_time[locale_][row[j]][row[k]] -= apper*weight
        weight = 0.
        for j in range(2,len(row)+1):
            if j >10:
                break
            for k in range(len(temp_list)):
                if j-1<= temp_list[k]:
                    weight = 1.0/pow(20,k)
                    break
            item_last_time[locale_][row[-j]][row[-1]] -= apper*weight
        #######################################################################################
        for j in range(1,len(prev_items)+1):
            add = Counter()
            add += c_plus(item_temp[locale_][prev_items[-j]],1)
            add += c_plus(item_next[locale_][prev_items[-j]],1)
            add += c_plus(item_next_last[locale_][prev_items[-j]],1)

            if j== 1:
                weight0 = 100
            elif j == 2:
                weight0 = 0.1
            elif j == 3:
                weight0 = 0.05
            elif j >= 4 and j <=5:
                weight0 = 0.05
            else:
                weight0 = 0.001

            temp += c_plus(add,weight0)

        temp2 = Counter()
        number_ = 10
        for item in temp.most_common(99999):
            if item[0] not in output and item[1]>0 and item[0] not in prev_items:
                if len(output)<160:
                    output.append(item[0])
                    number += 1
                    if item[0] == label:
                        index = number
                        break
                if number_!=0:
                    number_-=1
                    temp2 += c_plus(item_temp[locale_][item[0]], item[1])
                
        for item in temp2.most_common(40):
            if item[0] not in output and item[1]>0 and item[0] not in prev_items :
                output.append(item[0])
                number += 1
                if item[0] == label:
                    index = number
                    break
        temp = Counter()
        for j in range(1,len(prev_items)+1):
            add = Counter(item_last_time[locale_][prev_items[-j]])
            temp += add

        for item in temp.most_common(99999):
            if item[0] not in output and item[1]>0 and item[0] not in prev_items:
                output.append(item[0])
                number += 1
                if item[0] == label:
                    index = number
                    break
        temp = Counter()
        for j in range(1,len(prev_items)+1):
            add = Counter(item_next3[locale_][prev_items[-j]])
            temp += add

        for item in temp.most_common(99999):
            if item[0] not in output and item[1]>0 and item[0] not in prev_items:
                output.append(item[0])
                number += 1
                if item[0] == label:
                    index = number
                    break

        temp = Counter() 
        for j in range(1,len(prev_items)+1):#20 0.2693804550849466
            add = Counter()
            add += item_time[locale_][prev_items[-j]]
            if j in w_list:
                temp = c_plus(temp,20) + add
            else:
                temp += add

        for item in temp.most_common(99999):
            if item[0] not in output and item[1]>0 and item[0] not in prev_items:
                output.append(item[0])
                number += 1
                if item[0] == label:
                    index = number
                    break
        ##################################################################################################
        #Recover the impact of this session on the matrix
        apper=-1
        weight = 1
        if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
            weight=2
        item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
        for j in range(len(row)-1):
            item_next[locale_][row[j]][row[j+1]] -= apper
        check = Counter()
        for j in range(len(row)-3):
            if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
                continue
            check[(row[j],row[j+3])] = 1
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
                item_next3[locale_][row[j]][row[j+3]] -= apper
                item_next3_back[locale_][row[j+3]][row[j]] -= apper
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        check = Counter()
        for j in range(len(row)-1):
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
                item_temp[locale_][row[j]][row[j+1]] -= apper
                item_temp[locale_][row[j+1]][row[j]] -= apper
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        for j in range(len(row)-2):
            if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
                continue
            check[(row[j],row[j+2])] = 1
            if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
                item_temp[locale_][row[j]][row[j+2]] -= apper
                item_temp[locale_][row[j+2]][row[j]] -= apper
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        for j in range(len(row)):
            for k in range(len(row)):
                if j == k:
                    continue
                weight = 1
                if k<j:
                    weight *= 0.8
                if abs(k-j)>5:
                    weight *= 0.1
                elif abs(k-j)>=3:
                    weight *= 0.3
                elif abs(k-j)>=2:
                    weight *= 0.5
                if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                    weight *=2
                item_time[locale_][row[j]][row[k]] -= apper*weight
        weight = 0.
        for j in range(2,len(row)+1):
            if j >10:
                break
            for k in range(len(temp_list)):
                if j-1<= temp_list[k]:
                    weight = 1.0/pow(20,k)
                    break
            item_last_time[locale_][row[-j]][row[-1]] -= apper*weight
        
        output = output[:200]

        if index<=100:
            mrr += 1/index
        if index<=200:
            hit+=1
            
        elif index!=999999:
            check_+=1
            jz.append(index)
    print(mrr/len(valid_session))
    print(hit,check_)
    print(sum(jz)/len(jz))
mrr_()#0.3183140806783021 10838 363
# 0.319765
11116


# recall

## test

In [ ]:
#recall test
import copy
predictions = []


for i in tqdm(range(0,len(test_session))):
    output = []
    value = []
    locale_ = locale_map[test_locale[i]]
    prev_items = test_session[i]
    prev_items = list(dict.fromkeys(prev_items[::-1]))[::-1]
    
    w_list = [2,3,5]
    row = test_session[i]
########################################################################    
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter()
        add += c_plus(item_temp[locale_][prev_items[-j]],1)
        add += c_plus(item_next[locale_][prev_items[-j]],1)
        add += c_plus(item_next_last[locale_][prev_items[-j]],1)

        if j== 1:
            weight0 = 100
        elif j == 2:
            weight0 = 0.1
        elif j == 3:
            weight0 = 0.05
        elif j >= 4 and j <=5:
            weight0 = 0.05
        else:
            weight0 = 0.001

        temp += c_plus(add,weight0)

    temp2 = Counter()
    number_ = 15
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            if len(output)<160:
                output.append(item[0])

            if number_!=0:
                number_-=1
                temp2 += c_plus(item_temp[locale_][item[0]], item[1])
        
    for item in temp2.most_common(40):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items :
            output.append(item[0])
            
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter(item_last_time[locale_][prev_items[-j]])
        temp += add
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])
            
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter(item_next3[locale_][prev_items[-j]])
        temp += add
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])


    temp = Counter() 
    for j in range(1,len(prev_items)+1):#20 0.2693804550849466
        add = Counter()
        add += item_time[locale_][prev_items[-j]]
        if j in w_list:
            temp = c_plus(temp,20) + add
        else:
            temp += add

    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])
    ############################################################################# 
    
    output = output[:200]
    pred_item_asins = output
    predictions.append(pred_item_asins)
results = pd.DataFrame({'next_item_prediction':predictions})
results.to_pickle('recall1_test_200_p2.dataset')

## Rebuild the matrix

In [ ]:
""""
Eliminate the negative impact of data in the test dataset on the recall of the train dataset and the valid dataset
""""

In [18]:
apper = 1
item_next = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    for j in range(len(row)-1):
        item_next[locale_map[locale[i]]][row[j]][row[j+1]] += apper
item_next3 = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    check = Counter()
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] += apper
        item_next3[locale_][row[j]][row[j+3]] += apper

100%|██████████| 3272716/3272716 [00:14<00:00, 225986.05it/s]


In [19]:
item_next3_back = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    check = Counter()
    apper = 1
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3_back[locale_][row[j+3]][row[j]] += apper
        item_next3_back[locale_][row[j+3]][row[j]] += apper

100%|██████████| 3272716/3272716 [00:14<00:00, 225130.12it/s]


In [20]:

item_temp = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    check = Counter()
    locale_ = locale_map[locale[i]]
    apper = 1
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] += apper
            item_temp[locale_][row[j+1]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+1]] += apper
        item_temp[locale_][row[j+1]][row[j]] += apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = apper
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] += apper
            item_temp[locale_][row[j+2]][row[j]] += apper
        item_temp[locale_][row[j]][row[j+2]] += apper
        item_temp[locale_][row[j+2]][row[j]] += apper

100%|██████████| 3272716/3272716 [00:39<00:00, 82070.05it/s]


In [21]:

item_time = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    locale_ = locale_map[locale[i]]
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] += weight

100%|██████████| 3272716/3272716 [01:43<00:00, 31761.08it/s]


In [22]:
item_next_last = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(session))):
    row = session[i]
    locale_ = locale_map[locale[i]]
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] += weight

100%|██████████| 3272716/3272716 [00:04<00:00, 759647.63it/s]


## train

In [ ]:
# train
import copy
predictions = []
train_session = train_session[::10]
train_locale = train_locale[::10]
apper = 1
for i in tqdm(range(0,len(train_session))):
    output = []
    value = []
    locale_ = locale_map[train_locale[i]]
    prev_items = train_session[i][:-1]
    prev_items = list(dict.fromkeys(prev_items[::-1]))[::-1]
    label = train_session[i][-1]
    
    w_list = [2,3,5]
    # Remove the effect of this session on the matrix
    row = train_session[i]
    apper = 1
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
    for j in range(len(row)-1):
        item_next[locale_][row[j]][row[j+1]] -= apper
    check = Counter()
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        item_next3[locale_][row[j]][row[j+3]] -= apper
        item_next3_back[locale_][row[j+3]][row[j]] -= apper
    check = Counter()
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+1]] -= apper
        item_temp[locale_][row[j+1]][row[j]] -= apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+2]] -= apper
        item_temp[locale_][row[j+2]][row[j]] -= apper
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] -= apper*weight
    weight = 0.
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        item_last_time[locale_][row[-j]][row[-1]] -= apper*weight
    
########################################################################    
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter()
        add += c_plus(item_temp[locale_][prev_items[-j]],1)
        add += c_plus(item_next[locale_][prev_items[-j]],1)
        add += c_plus(item_next_last[locale_][prev_items[-j]],1)

        if j== 1:
            weight0 = 100
        elif j == 2:
            weight0 = 0.1
        elif j == 3:
            weight0 = 0.05
        elif j >= 4 and j <=5:
            weight0 = 0.05
        else:
            weight0 = 0.001

        temp += c_plus(add,weight0)

    temp2 = Counter()
    number_ = 15
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            if len(output)<160:
                output.append(item[0])

            if number_!=0:
                number_-=1
                temp2 += c_plus(item_temp[locale_][item[0]], item[1])
        
    for item in temp2.most_common(40):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items :
            output.append(item[0])
            
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter(item_next3[locale_][prev_items[-j]])
        temp += add
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])


    temp = Counter() 
    for j in range(1,len(prev_items)+1):#20 0.2693804550849466
        add = Counter()
        add += item_time[locale_][prev_items[-j]]
        if j in w_list:
            temp = c_plus(temp,20) + add
        else:
            temp += add

    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])
    #############################################################################
    # Add the effect of this session on the matrix
    apper = -1
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
    for j in range(len(row)-1):
        item_next[locale_][row[j]][row[j+1]] -= apper
    check = Counter()
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        item_next3[locale_][row[j]][row[j+3]] -= apper
        item_next3_back[locale_][row[j+3]][row[j]] -= apper
    check = Counter()
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+1]] -= apper
        item_temp[locale_][row[j+1]][row[j]] -= apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+2]] -= apper
        item_temp[locale_][row[j+2]][row[j]] -= apper
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] -= apper*weight
    weight = 0.
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        item_last_time[locale_][row[-j]][row[-1]] -= apper*weight

    
    output = output[:200]
    pred_item_asins = output
    predictions.append(pred_item_asins)
recall_train = pd.DataFrame({'next_item_prediction':predictions})

In [24]:
recall_train.to_pickle('recall1_train_200_p2.dataset')

## Valid

In [34]:
#valid
import copy
predictions = []
apper = 1
for i in tqdm(range(len(valid_session))):
    output = []
    value = []
    locale_ = locale_map[valid_locale[i]]
    prev_items = valid_session[i][:-1]
    prev_items = list(dict.fromkeys(prev_items[::-1]))[::-1]
    label = valid_session[i][-1]
    temp = Counter()
    w_list = [2,3,5]
    # Remove the effect of this session on the matrix
    row = valid_session[i]
    apper = 1
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
    for j in range(len(row)-1):
        item_next[locale_][row[j]][row[j+1]] -= apper
    check = Counter()
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        item_next3[locale_][row[j]][row[j+3]] -= apper
        item_next3_back[locale_][row[j+3]][row[j]] -= apper
    check = Counter()
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+1]] -= apper
        item_temp[locale_][row[j+1]][row[j]] -= apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+2]] -= apper
        item_temp[locale_][row[j+2]][row[j]] -= apper
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] -= apper*weight
    weight = 0.
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        item_last_time[locale_][row[-j]][row[-1]] -= apper*weight
    ########################################################################    
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter()
        add += c_plus(item_temp[locale_][prev_items[-j]],1)
        add += c_plus(item_next[locale_][prev_items[-j]],1)
        add += c_plus(item_next_last[locale_][prev_items[-j]],1)

        if j== 1:
            weight0 = 100
        elif j == 2:
            weight0 = 0.1
        elif j == 3:
            weight0 = 0.05
        elif j >= 4 and j <=5:
            weight0 = 0.05
        else:
            weight0 = 0.001

        temp += c_plus(add,weight0)

    temp2 = Counter()
    number_ = 15
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            if len(output)<160:
                output.append(item[0])

            if number_!=0:
                number_-=1
                temp2 += c_plus(item_temp[locale_][item[0]], item[1])
        
    for item in temp2.most_common(40):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items :
            output.append(item[0])
            
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter(item_last_time[locale_][prev_items[-j]])
        temp += add
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])
            
    temp = Counter()
    for j in range(1,len(prev_items)+1):
        add = Counter(item_next3[locale_][prev_items[-j]])
        temp += add
    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])


    temp = Counter() 
    for j in range(1,len(prev_items)+1):#20 0.2693804550849466
        add = Counter()
        add += item_time[locale_][prev_items[-j]]
        if j in w_list:
            temp = c_plus(temp,20) + add
        else:
            temp += add

    for item in temp.most_common(99999):
        if item[0] not in output and item[1]>0 and item[0] not in prev_items:
            output.append(item[0])
    #############################################################################       
    # Add the effect of this session on the matrix
    apper = -1
    weight = 1
    if brand_map[locale_][row[-1]] == brand_map[locale_][row[-2]]:
        weight=2
    item_next_last[locale_][row[-2]][row[-1]] -= weight*apper
    for j in range(len(row)-1):
        item_next[locale_][row[j]][row[j+1]] -= apper
    check = Counter()
    for j in range(len(row)-3):
        if check[(row[j],row[j+3])] == 1 or row[j] == row[j+3]:
            continue
        check[(row[j],row[j+3])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+3]]:
            item_next3[locale_][row[j]][row[j+3]] -= apper
            item_next3_back[locale_][row[j+3]][row[j]] -= apper
        item_next3[locale_][row[j]][row[j+3]] -= apper
        item_next3_back[locale_][row[j+3]][row[j]] -= apper
    check = Counter()
    for j in range(len(row)-1):
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+1]]:
            item_temp[locale_][row[j]][row[j+1]] -= apper
            item_temp[locale_][row[j+1]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+1]] -= apper
        item_temp[locale_][row[j+1]][row[j]] -= apper
    for j in range(len(row)-2):
        if check[(row[j],row[j+2])] == 1 or row[j] == row[j+2]:
            continue
        check[(row[j],row[j+2])] = 1
        if brand_map[locale_][row[j]] == brand_map[locale_][row[j+2]]:
            item_temp[locale_][row[j]][row[j+2]] -= apper
            item_temp[locale_][row[j+2]][row[j]] -= apper
        item_temp[locale_][row[j]][row[j+2]] -= apper
        item_temp[locale_][row[j+2]][row[j]] -= apper
    for j in range(len(row)):
        for k in range(len(row)):
            if j == k:
                continue
            weight = 1
            if k<j:
                weight *= 0.8
            if abs(k-j)>5:
                weight *= 0.1
            elif abs(k-j)>=3:
                weight *= 0.3
            elif abs(k-j)>=2:
                weight *= 0.5
            if brand_map[locale_][row[j]] == brand_map[locale_][row[k]]:
                weight *=2
            item_time[locale_][row[j]][row[k]] -= apper*weight
    weight = 0.
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        item_last_time[locale_][row[-j]][row[-1]] -= apper*weight
    
    output = output[:200]
    pred_item_asins = output
    predictions.append(pred_item_asins)
recall_valid = pd.DataFrame({'next_item_prediction':predictions})

100%|██████████| 16364/16364 [01:27<00:00, 187.93it/s]


In [27]:
recall_valid.to_pickle('recall1_valid_200_p2.dataset')